In [1]:
%========================================================================%
% GPPI_WRAPPER
%
%   This script allows you to run Donald McLaren's gPPI toolbox to build 
%   and estimate psychophysiological interaction (PPI) models for multiple
%   subjects and for multiple seed volumes-of-interest (VOI). Note that
%   before using this, you must first create image files corresponding to
%   your seed regions. 

%  SECTIONS REQUIRING INPUT:
%  * Paths to input and output directories and models
%  * ROIs [earlier version of this code required all ROIs to be in the same directory; this does not, though it does require manual listing of ROI paths and names]
%  * Subjects
%  * Defining conditions in the model - - make sure that you include ALL task conditions / regressors in this list. Code is provided to read out regressor names from first level SPM.mat files.
%  * Edit contrasts in LOOP section (do not need to edit other settings in LOOP)
%
%   History
%   Feb 8, 2020 - NC adapting for DARPA w/ RP and EB; using most recent version of gPPI toolbox (PPPIv13) and updated notes from McClaren and Mumford
%       Moving away from Bob Spunt's wrapper (as in data00/projects/socnets_darpa2/scripts/PPPIv13_test_nc.ipynb); now just using SPM toolbox
%=========================================================================%

In [25]:
clear all; home;

gPPIpath='/data00/tools/PPPIv13.1';
addpath(genpath(gPPIpath)); % path for gPPI toolbox
addpath(genpath('/data00/tools/spm12')) 

rmpath('/data00/tools/batch8');
rmpath('/data00/tools/spm8');


> In rmpath (line 71)
> In rmpath (line 71)


In [26]:
%---------PATHS TO INPUT AND OUTPUT DIRECTORIES AND MODELS------%
studypath = '/data00/projects/socnets_darpa2/data/subjs'; % path for study directory
level1name = '/model_APPRATE_bCHANGE_direction_review_only/'; % path of level1 analysis (relative to subject folder)
ppi_folder_affix = 'PPItest0208';
%voi_save_dir = ['/data00/projects/socnets_darpa2/data/ppi_models' level1name '/VOI_info']; % directory for saving analysis and ROI specific analysis specification files 


In [29]:
% -----------ROIs-----------
% Edit "regionfile" and "regions" lists

%regionfile={'/data00/projects/socnets_darpa1/data/ROIs/PPI_ROIs/kable_vmpfc.nii'};
regionfile={'/data00/projects/socnets_darpa1/data/ROIs/PPI_ROIs/reward/reward_neurosynth.nii'};
regions={'reward_neurosynth'};

%for i=1:length(regions)
  %Set the location to save the region specific mat-files
%    save([voi_save_dir filesep regions{i} '.mat'],'P');   
%end

nmasks = length(regions);
domasks = 1:nmasks;


In [30]:
%-----------Subjects ------------------
% Edit subjectpattern and subTAG

%subjectpattern = 'DP*'; % pattern for finding subject folders (use wildcards)
subjectpattern = 'DP257*';
subTAG = 'all'; 

%%subTAG = 'all'; %  do which subjects? 'all' to do all, position indices to do subset, e.g., [1 3 7]
%subTAG = [ 1 2 3 4 5 6 8 9 10 11 12 13 14 15 16 17 18 20 21 22 23 24 26 27 28 29 30 32 33 34 37 38 39 40 ]; %exclude 7 & 19/ %25,31,35,36 do not have run 3s

cd(studypath);
fprintf('\nSUBJECT LIST:\n');
d=dir(subjectpattern);
for i=1:length(d)
    subnam{i}=d(i).name;
    subdir{i} = [studypath '/' subnam{i}];
    fprintf('\tAdding %s to subject list\n',subnam{i})
end
nsubs = length(subnam);
if strcmp(subTAG,'all')
    dosubs = 1:nsubs;
else
    dosubs = subTAG;
end


SUBJECT LIST:
	Adding DP257 to subject list


In [31]:
%-------- Defining conditions in the model -------%
% Visually confirm that the correct conditions are specified
% -- This code pulls the regressor list from the first run of the first subject
% -- Ensure that there are no conditions missing
% -- Use this list to create the conditions list in the next cell

load([[studypath, '/', subnam{1}, '/model/' level1name,'SPM.mat']]) 
SPM.Sess(1).U.name




ans =

  1x1 cell array

    {'bCHANGEcHIGHER'}


ans =

  1x1 cell array

    {'bCHANGEcLOWER'}


ans =

  1x1 cell array

    {'bCHANGEncHIGHER'}


ans =

  1x1 cell array

    {'bNOCHANGEgHIGHER'}


ans =

  1x1 cell array

    {'bNOCHANGEgLOWER'}


ans =

  1x1 cell array

    {'zIGNORE'}



In [32]:
% Set condition list here - use ALL regressors 
conditions = { 'bCHANGEcHIGHER' 'bCHANGEcLOWER' 'bCHANGEncHIGHER' 'bCHANGEncLOWER' 'bNOCHANGEgHIGHER' 'bNOCHANGEgLOWER'}; % open SPM.mat file, Sess to see the regressor names. conditions to compute PPIs for (must match names used in level 1 analysis)


In [33]:
% LOOP OVER ALL SUBJECTS AND REGIONS TO RUN PPPI

for ii=1:numel(dosubs)
    for jj=1:numel(regions)
        cd(gPPIpath)
        clear P 
        load('/data00/tools/PPPIv13.1/PPPIv13/parameters.mat'); % load parameters template
%----------EDIT CONTRASTS------------%
        P.Contrasts(1).name = 'bCHANGEcHIGHERvsbNOCHANGE';
        P.Contrasts(1).left = {'bCHANGEcHIGHER'};
        P.Contrasts(1).right = {'bNOCHANGEgHIGHER' 'bNOCHANGEgLOWER'}; %right is the one that gets subtracted - if you want it to be rest put none

        %P.Contrasts(2) = P.Contrasts(1);
        %P.Contrasts(2).name = 'bCHANGEcLOWERvsbNOCHANGE';
        %P.Contrasts(2).left = {'bCHANGEcLOWER'};
        %P.Contrasts(2).right = {'bNOCHANGEgHIGHER' 'bNOCHANGEgLOWER'}; %right is the one that gets subtracted - if you want it to be rest put none


%------------------------------------%
    

        P.subject=subnam{ii};
        P.VOI.VOI = [regionfile{jj}];%filepath and name of the VOI
        P.VOI.masks = {'mask.img'}; %images to specify the subject specific ROI; % name of images to use to constrain definition of the seed region; default here is the mask.img file within the first level model
        P.VOI.thresh = 0.5; %threshold (statistic value) for the VOI
        P.VOI.min = 10; %minimum VOI size required
        P.VOImin = 10; %minimum VOI size required
        P.Region = [regions{jj} '_' ppi_folder_affix]; %string containing the basename of the output file(s)
        P.directory=[studypath '/' subnam{ii} '/model/' level1name]; %path to the first level SPM directory
        P.Estimate = 1; %estimate design (2 means already estimated, 0 means do not estimate)
        P.contrast = 1; %F contrast to adjust for (corresponds to number of the ess image in level 1 folder)
        P.extract='eig'; % extract timecourse as first eigenvariate ('eig') or mean  ('mean')
        P.Tasks = ['0' conditions]; %In the generalized context-dependent PPI, you need specify the tasks to include in the analyses, but put a ‘0’ or ‘1’ in front of them to specify if they must exist in all sessions
        P.Weights=[]; %only relevant for traditional PPI
        P.maskdir = []; %location to store VOI file if input VOI was a mat file
        P.equalroi = 0; %specifies that the ROIs must be the same size in all subjects (1 for true, 0 for false)
        P.FLmask = 1; %specifies that ROI should be restricted in each subject by the mask image (1 for true, 0 for false)
        P.analysis = 'psy'; %specifies psychophysiological interaction ('psy'), physiophysiological interaction ('phys'), or psychophysiophysiological interaction ('psyphy')
        P.method = 'cond'; %specifies traditional spm ppi ('trad') or generalized context-dependent ppi ('cond')
        P.CompContrasts = 1; %estimate contrasts
        P.Weighted = 0; %default is to not weight tasks by number of trials (0); to change this, specify which tasks should be weighted by trials
   
        spmfile=[studypath, '/', subnam{ii}, '/model/' level1name,'SPM.mat'];
        load(spmfile)
        
        PPPI(P);

    end
end
        

Log File: DP257_PPPI_2_9_2021_2.log
 
PPPI Version: 13.1.4-17-2014
 
 
Parameters used:
     Processing subject: DP257
     VOI file          : /data00/projects/socnets_darpa1/data/ROIs/PPI_ROIs/reward/reward_neurosynth.nii
     Output file will be: DP257_reward_neurosynth_PPItest0208_session#_cond_PPI_regressors.txt
     Contrast          : fcon
     Analysis          : Psychophysiological Interactions
     Extraction        : eigenvariate
     Tasks             : _0_bCHANGEcHIGHER_bCHANGEcLOWER_bCHANGEncHIGHER_bCHANGEncLOWER_bNOCHANGEgHIGHER_bNOCHANGEgLOWER
     Method            : Condition Specific
> In spm_jobman (line 106)
  In timeseries_extract (line 154)
  In PPPI (line 462)
Item 'Deformations', field 'prog': Value must be a function or function handle on MATLAB path.
Duplicate application tag in applications 'SPM' and 'SPM'.
Duplicate application tag in applications 'BasicIO' and 'BasicIO'.


------------------------------------------------------------------------
Running job